In [1]:
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from pymongo import MongoClient

In [2]:
chrome_options = Options()
chrome_options.add_argument('--window-size=1920,1080')

In [3]:
class SeleniumParser(object):
    main_url = ''
    table_name = ''

    def __init__(self, driver_path='./chromedriver'):
        self.driver = webdriver.Chrome(driver_path, options=chrome_options)
        self.driver.get(self.main_url)
        client = MongoClient('localhost', 27017)
        self.mongobase = client.selenium_parsed_data

    def save_data_to_mongo(self, data):
        collection = self.mongobase[self.table_name]
        collection.insert_many(data)

    def quit(self):
        self.driver.quit()


In [4]:
class MailruParser(SeleniumParser):
    main_url = 'https://m.mail.ru/'
    table_name = 'emails'

    def authorize(self, login, password):
        button = self.driver.find_element_by_class_name('social__title')
        button.click()

        login_elem = self.driver.find_element_by_name('Login')
        login_elem.send_keys(login)

        password_elem = self.driver.find_element_by_name('Password')
        password_elem.send_keys(password)

        button = self.driver.find_element_by_xpath("//input[@value='Войти']")
        button.click()

    def get_email_data(self, email):
        href = email.find_element_by_class_name('messageline__link').get_attribute('href')
        self.driver.execute_script("window.open()")
        self.driver.switch_to.window(self.driver.window_handles[1])
        self.driver.get(href)

        reply_btn = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located(
                (By.CLASS_NAME, "readmsg__reply-type_forward"))
        )
        reply_btn.click()

        text = self.driver.find_element_by_tag_name('textarea').text
        author = text[text.find('От кого: ') + len('От кого: '): text.find('Кому: ') - 1]
        date = text[text.find('Дата: ') + len('Дата: '): text.find('Тема: ') - 1]
        topic = text[text.find('Тема: ') + len('Тема: '): text.find('\n', text.find('Тема: ') + len('Тема: '))]
        text = text[text.find('\n', text.find('Тема: ') + len('Тема: ')):]
        self.driver.close()
        self.driver.switch_to.window(self.driver.window_handles[0])

        return {'author': author,
                'topic': topic,
                'date': date,
                'text': text}

    def get_emails(self):
        emails_data = []
        while True:
            emails_block = self.driver.find_elements_by_class_name('messageline')
            for email in emails_block:
                msg_data = self.get_email_data(email)
                emails_data.append(msg_data)
            try:
                next_page = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//a[@title='Следующая страница']"))
                )
                next_page.click()
            except:
                break
        return emails_data

In [5]:
class MvideoParser(SeleniumParser):
    main_url = 'https://www.mvideo.ru/'
    table_name = 'sales_hits'

    def get_product_data(self, product):
        data = product.get_attribute('data-product-info')
        data = json.loads(data)
        return data

    def get_hits(self):
        hits_block = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'sel-hits-block'))
        )
        while True:
            try:
                next_page = WebDriverWait(hits_block, 10).until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, 'sel-hits-button-next'))
                )
                if 'disabled' in next_page.get_attribute('class'):
                    break
                else:
                    next_page.click()
            except:
                break

        data = []

        hits_block = hits_block.find_elements_by_class_name('gallery-list-item')
        for hit in hits_block:
            hit = hit.find_element_by_class_name('sel-product-tile-title')
            product = self.get_product_data(hit)
            data.append(product)

        return data

In [6]:
import pandas
import constants

In [7]:
mailru = MailruParser()

mailru.authorize(constants.LOGIN, constants.PASSWORD)

data = mailru.get_emails()
mailru.save_data_to_mongo(data)

mailru.quit()

In [8]:
df = pandas.DataFrame(data)
df

,author,topic,date,text,_id
0,"""GeekBrains"" <digest@e.geekbrains.ru>",⚡⚡⚡ Новый факультет!,"Среда, 1 апреля 2020, 16:00 +03:00",\n\n\nНабор начинается уже с сегодняшнего дня....,5e85dc3d585be53847157a66
1,"""Mail.ru"" <security@id.mail.ru>",Вход с нового устройства в аккаунт,"Среда, 1 апреля 2020, 12:27 +03:00","\n\n\nЕсли это не вы, измените пароль. \n \n \...",5e85dc3d585be53847157a67
2,"""Mail.ru"" <security@id.mail.ru>",Вход с нового устройства в аккаунт,"Среда, 1 апреля 2020, 12:25 +03:00","\n\n\nЕсли это не вы, измените пароль. \n \n \...",5e85dc3d585be53847157a68
3,"""Почта Mail.ru"" <team@e.mail.ru>",Поздравляем с днём рождения!,"Среда, 1 апреля 2020, 9:24 +03:00",\n\n\nУ нас для вас много подарков! Открывайте...,5e85dc3d585be53847157a69
4,"""GeekBrains"" <digest@e.geekbrains.ru>",13 ошибок программиста-новичка,"Четверг, 26 марта 2020, 11:31 +03:00",\n\n\nСтрашное письмо со счастливым концом. \n...,5e85dc3d585be53847157a6a
...,...,...,...,...,...
259,"""Tproger"" <news@tproger.ru>",Вечерний программист,"Пятница, 12 июля 2019, 11:54 +03:00",\n\nСпасибо за подписку!\nАргументы и функции\...,5e85dc3d585be53847157b69
260,"""Tproger Live"" <news@tproger.ru>",Аргументы и функции: подтверждение подписки,"Пятница, 12 июля 2019, 11:51 +03:00","\n\n\nПожалуйста, подтвердите подписку Подтвер...",5e85dc3d585be53847157b6a
261,"""Команда Почты Mail.ru"" <welcome@corp.mail.ru>","Mail.ru – больше, чем почта. Познакомьтесь с п...","Пятница, 12 июля 2019, 11:48 +03:00","\n\n\n \n \nЗдравствуйте, study!\n \n \n \nMa...",5e85dc3d585be53847157b6b
262,"""Команда Почты Mail.ru"" <welcome@corp.mail.ru>",Как воспользоваться почтой с мобильного?,"Пятница, 12 июля 2019, 11:48 +03:00","\n\n\n \nЗдравствуйте, study!\n \n \n \nПольз...",5e85dc3d585be53847157b6c


In [9]:
mvideo = MvideoParser()

data = mvideo.get_hits()

mvideo.save_data_to_mongo(data)
mvideo.quit()

In [10]:
df = pandas.DataFrame(data)
df

,productPriceLocal,productId,productName,productCategoryId,productCategoryName,productVendorName,productGroupId,Location,eventPosition,_id
0,2490.00,50129349,Наушники Bluetooth JBL Tune 590BT Blue,cat2_cis_0000000728,Наушники,JBL,cat1_cis_0000000155,block5260655,5,5e85dc46585be53847157b6f
1,2790.00,20036553,Утюг Philips GC3580/20,cat2_cis_0000000052,Утюги,Philips,cat1_cis_0000000002,block5260655,6,5e85dc46585be53847157b70
2,17990.00,10013407,Беспроводная акустика JBL Boombox Black,cat2_cis_0000000341,Портативная акустика,JBL,cat1_cis_0000000115,block5260655,7,5e85dc46585be53847157b71
3,16990.00,20059065,Газовая плита (50-55 см) Hansa FCGW510977,cat2_cis_0000000111,Плиты газовые и электрические,Hansa,cat1_cis_0000000135,block5260655,8,5e85dc46585be53847157b72
4,4990.00,10018338,Беспроводная акустика Sony SRS-XB21/BC,cat2_cis_0000000341,Портативная акустика,Sony,cat1_cis_0000000115,block5260655,5,5e85dc46585be53847157b73
5,2990.00,20038529,Фен Rowenta Premium Care Pro CV7461F0,cat2_cis_0000000757,Фены и фен-щетки,Rowenta,cat1_cis_0000000204,block5260655,6,5e85dc46585be53847157b74
6,1590.00,50132280,Игровая клавиатура A4Tech B120N Black,cat2_cis_0000000378,Клавиатуры и комплекты,A4Tech,cat1_cis_0000000114,block5260655,7,5e85dc46585be53847157b75
7,9990.00,20063292,Пылесос ручной (handstick) Tefal Air Force 160...,cat2_cis_0000000639,Пылесосы,Tefal,cat1_cis_0000000184,block5260655,8,5e85dc46585be53847157b76
8,19990.00,30047894,Смартфон Samsung Galaxy A51 64GB Black (SM-A515F),cat2_cis_0000000357,Смартфоны,Samsung,cat1_cis_0000000012,block5260655,1,5e85dc46585be53847157b77
9,17990.00,20032635,Стиральная машина узкая Hotpoint-Ariston RSM 6...,cat2_cis_0000000049,Стиральные машины,Hotpoint-Ariston,cat1_cis_0000000183,block5260655,2,5e85dc46585be53847157b78
